# Playground

In [1089]:
import pandas as pd
import feather
import pycountry
import dash
from dash.dependencies import Input, Output
import plotly.express as px
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go


In [1090]:
# load feather file from input directory
df = pd.read_feather('../input/df_allyears')

In [1091]:
def alpha2_to_alpha3(alpha_2):
    '''Converts ISO 2 country code to ISO 3 country code'''
    country = pycountry.countries.get(alpha_2=alpha_2)
    return country.alpha_3 if country else alpha_2

In [1092]:
## Adds country name to dataframe
country_mapping = {country.alpha_3: country.name for country in pycountry.countries}

In [1093]:
df['Country'] = df['Country'].apply(alpha2_to_alpha3)

In [1094]:
df['Country_Name'] = df['Country'].map(country_mapping)

In [1095]:
not_converted = df[df['Country'].str.len() == 2]['Country'].unique()
print(not_converted)

# Drop rows where 'Country' column contains 2-letter codes
df = df[df['Country'].str.len() != 2]

['AN' 'YU' 'CS' 'DD' 'OA' 'UK' 'EP']


In [1096]:
# rename column earlist_year to year
df.rename(columns={'earliest_filing_year': 'Year'}, inplace=True)

In [1097]:
# Retrieves alle ISO 3 country codes and country names from pycountry
countries_df = pd.DataFrame({
    'Country': [country.alpha_3 for country in pycountry.countries],
    'Country_Name': [country.name for country in pycountry.countries]
})

In [1098]:
# create a DataFrame that includes all years in your original dataset
years_df = pd.DataFrame({'Year': df['Year'].unique()})

In [1099]:
# create a DataFrame that includes every combination of country and year.
countries_years_df = pd.merge(
    countries_df.assign(key=0),
    years_df.assign(key=0),
    on='key'
).drop('key', axis=1)

In [1100]:
# merge this countries_years_df DataFrame with original df 
df = pd.merge(countries_years_df, df, on=['Country', 'Country_Name', 'Year'], how='outer')


In [1101]:
# Replace NaN values with 0
df['ff'] = df['ff'].fillna(0)
df['renewables'] = df['renewables'].fillna(0)
df['share_renewables'] = df['share_renewables'].fillna(0)
df['share_ff'] = df['share_ff'].fillna(0)


In [1102]:
# Keep only data from the year 2000 to 2017
df = df[df['Year'] >= 2000]
df = df[df['Year'] <= 2017]


In [1103]:
# Round share_ff and share_renewables to 2 decimals
df['share_ff'] = df['share_ff'].round(2)
df['share_renewables'] = df['share_renewables'].round(2)

In [1104]:
# Custom color scale
color_scale_renewables = [
    (0.0, 'rgb(211,211,211)'),  # Light grey for 0
    (0.00001, 'rgb(229,245,224)'),  # Start shades of green for any value higher than 0
    (0.1, 'rgb(199,233,192)'),
    (0.2, 'rgb(161,217,155)'),
    (0.3, 'rgb(116,196,118)'),
    (0.4, 'rgb(65,171,93)'),
    (0.5, 'rgb(35,139,69)'),
    (0.6, 'rgb(0,109,44)'),
    (0.7, 'rgb(0,68,27)'),
    (0.8, 'rgb(0,68,27)'),
    (0.9, 'rgb(0,68,27)'),
    (1.0, 'rgb(0,68,27)')  # Dark green for all values above 150
]

color_scale_ff = [
    (0.0, 'rgb(211,211,211)'),  # Light grey for 0
    (0.00001, 'rgb(246,232,195)'),  # Very light beige
    (0.1, 'rgb(223,194,125)'),
    (0.2, 'rgb(191,129,45)'),
    (0.3, 'rgb(141,109,49)'),
    (0.4, 'rgb(140,81,10)'),
    (0.5, 'rgb(129,89,23)'),
    (0.6, 'rgb(103,65,15)'),
    (0.7, 'rgb(80,50,20)'),
    (0.8, 'rgb(60,38,22)'),
    (0.9, 'rgb(40,25,15)'),
    (1.0, 'rgb(20,10,5)')  # Dark brown for all values above 150
]

# app = dash.Dash(__name__)
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.Header([
        html.H1("Green Energy Innovation"),
        html.H5("Work by Theo, Ana, Carole"),
    ]),
    html.Main([
        html.P("Welcome to our page! Here, you can find different world maps that show the number of patents filed for renewable energy and fossil fuel energy innovations. You can also see the share of renewable energy innovations in the total number of innovations, and the share of fossil fuel energy innovations in the total number of innovations. You can select the year you want to see by using the slider below the maps. Scroll down for more information on data sources and variables!"),

        dcc.Dropdown(
            id="type-selection", 
            options=[
                {"label": "Fossil Fuel Energy Innovation", "value": 'ff'},
                {"label": "Renewable Energy Innovation", "value": 'renewables'},
                {"label": "Share of Fossil Fuel Innovations", "value": 'share_ff'},
                {"label": "Share of Renewable Energy Innovations", "value": 'share_renewables'}
            ],
            value='ff',
            clearable=False,
            className = "dropdown"
        ),
        dcc.Slider(
            id='year-slider',
            min=df['Year'].min(),
            max=df['Year'].max(),
            value=df['Year'].max(), # Set last year as the default value
            marks={str(year): str(year) for year in df['Year'].unique()},
            step=None
        ),
        dcc.Graph(id="world-map", style={"height": "800px", "width": "100%"}),
        dbc.Modal([
            dbc.ModalHeader(id="modal-header"),
            dbc.ModalBody(dcc.Graph(id='country-trend')),
        ], id="modal", is_open=False),
]),
    html.Footer([
        html.Div([
            html.H4('Data Sources', className = 'footer-heading'),
            html.P('PATSAT bla bla bla', className = 'footer-text'),
        ], className = 'footer-column'),

        html.Div([
            html.H4('Explanation of Variables', className = 'footer-heading'),
            html.P('Fossil Fuel Energy Innovation: bla bla bla'),
            html.P('Renewable Energy Innovation: bla bla bla'),
            html.P('Share of Fossil Fuel Innovations: bla bla bla'),
            html.P('Share of Renewable Energy Innovations: bla bla bla'),
        ], className = 'footer-column'),
    ])

])


@app.callback(
    Output("world-map", "figure"), 
    [Input("type-selection", "value"), Input('year-slider', 'value')]
)
def update_world_map(energy_type, selected_year):
    dff = df[df['Year'] == selected_year]

    if energy_type == "ff":
        color_scale = color_scale_ff  # brownish color scale for fossil fuels
    elif energy_type == "renewables":
        color_scale = color_scale_renewables  # custom green color scale for renewables
    elif energy_type == "share_ff":
        color_scale = color_scale_ff  
    elif energy_type == "share_renewables":
        color_scale = color_scale_renewables  
    else:
        color_scale = px.colors.sequential.Plasma  # default color scale, you can choose any color scale here

    # Define the label based on the energy type
    if energy_type in ["share_ff", "share_renewables"]:
        label = 'Share of Energy Patents'
    else:
        label = 'Number of Patents'

    fig = px.choropleth(
        dff, 
        locations="Country", 
        color=energy_type, 
        hover_name="Country_Name", 
        projection="natural earth",
        color_continuous_scale=color_scale,
        labels={energy_type: label}  # use the dynamic label here
    )

    fig.update_layout(clickmode='event+select')  # Enable click events
    return fig

@app.callback(
    [Output("modal", "is_open"), Output("country-trend", "figure"), Output("modal-header", "children")],
    [Input("world-map", "clickData"), Input("type-selection", "value")],
    [State("modal", "is_open")]
)
def update_country_trend(clickData, energy_type, is_open):
    if clickData is None:
        return is_open, go.Figure(), "Country Trends"

    country_code = clickData['points'][0]['location']
    dff = df[df.Country == country_code]
    country_name = dff['Country_Name'].values[0]

    energy_label_dict = {
        'ff': 'Fossil Fuel Energy Innovation', 
        'renewables': 'Renewable Energy Innovation', 
        'share_ff': 'Share of Fossil Fuel Innovations', 
        'share_renewables': 'Share of Renewable Energy Innovations'
    }
    energy_label = energy_label_dict[energy_type]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dff["Year"], y=dff[energy_type], mode='lines+markers', name=energy_label))
    fig.update_layout(title_text="Time Trend of {}".format(energy_label))
    
    return not is_open, fig, "Time Trend of {} for {}".format(energy_label, country_name)

if __name__ == '__main__':
    app.run_server(debug=True)
    print("The app is running at: http://localhost:8050")


The app is running at: http://localhost:8050
